In [1]:
import sys
import collections
import mxnet as mx
from mxnet import autograd, gluon, init, metric, nd
from mxnet.gluon import loss as gloss, nn, rnn
from mxnet.contrib import text
import os
import random
import zipfile
from sklearn.model_selection import train_test_split
import spacy
import time
from time import strftime
from gensim import corpora
from gensim import models

D:\Anaconda3\envs\gluon\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
D:\Anaconda3\envs\gluon\lib\site-packages\scipy\sparse\sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
ROOT_PATH  = "data/"

In [3]:
def noation_replace(line):
    notations = ['?', '¿', ',', '.', '¡','!', ':', ",", ';', '-', '']
    output = line.strip()
    for i in notations:
        output = output.replace(i, '')
    return output.split('\t')

In [4]:
es_e_l = []
es_s_l = []
es_e_r = []
es_s_r = []
es_labels = []
## english-spanish text 
with open(os.path.join(ROOT_PATH, "cikm_english_train_20180516.txt"), 'r', encoding='utf-8') as esf:
    for line in esf:
        segs = noation_replace(line)
        es_e_l.append(segs[0].lower())
        es_e_r.append(segs[2].lower())
        es_s_l.append(segs[1].lower())
        es_s_r.append(segs[3].lower())
        es_labels.append(int(segs[4]))
        
se_e_l = []
se_s_l = []
se_e_r = []
se_s_r = []
se_labels = []
## spanish-english text
with open(os.path.join(ROOT_PATH, "cikm_spanish_train_20180516.txt"), 'r', encoding='utf-8') as ssf:
    for line in ssf:
        segs = noation_replace(line)
        se_s_l.append(segs[0].lower())
        se_s_r.append(segs[2].lower())
        se_e_l.append(segs[1].lower())
        se_e_r.append(segs[3].lower())
        se_labels.append(int(segs[4]))

test_s_1 = []
test_s_2 = []
## spanish test file
with open(os.path.join(ROOT_PATH, "cikm_test_a_20180516.txt"), 'r', encoding='utf-8') as tef:
    for line in tef:
        segs = noation_replace(line)
        test_s_1.append(segs[0].lower())
        test_s_2.append(segs[1].lower())

print("es data size:", len(es_s_l))
print("se data size:", len(se_e_l))
print("test data size:", len(test_s_1))

es data size: 20000
se data size: 1400
test data size: 5000


In [5]:
##add data sets (s0, s1, y) + (s1, s0, y)
left_texts = es_s_l + se_s_l
right_texts = es_s_r + se_s_r
y = es_labels + se_labels

print("left data size:", len(left_texts))
print("right data size:", len(right_texts))
print("label size:", len(y))

left data size: 21400
right data size: 21400
label size: 21400


In [6]:
span_sw = []
with open(os.path.join(ROOT_PATH, "spanish.txt"), 'r', encoding='utf-8') as swf:
    for line in swf:
        word = line.strip()
        span_sw.append(word)

In [7]:
spacy_en = spacy.load('en')
spacy_es = spacy.load('es')

def en_tokenizer(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

def es_tokenizer(text):
    return [tok.text for tok in spacy_es.tokenizer(text) if tok.text not in span_sw]

D:\Anaconda3\envs\gluon\lib\site-packages\msgpack_numpy.py:84: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr)).reshape(obj[b'shape'])
D:\Anaconda3\envs\gluon\lib\site-packages\msgpack_numpy.py:88: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  dtype=np.dtype(descr))[0]


In [8]:
left_tokenized = []
right_tokenized = []
test_left_tokenized = []
test_right_tokenized = []

for token in left_texts:
    left_tokenized.append(es_tokenizer(token))

for token in right_texts:
    right_tokenized.append(es_tokenizer(token))
    
for token in test_s_1:
    test_left_tokenized.append(es_tokenizer(token))

for token in test_s_2:
    test_right_tokenized.append(es_tokenizer(token))
    
    
token_counter = collections.Counter()
for sample in left_tokenized:
    for token in sample:
        if token not in token_counter:
            token_counter[token] = 1
        else:
            token_counter[token] += 1
            
for sample in right_tokenized:
    for token in sample:
        if token not in token_counter:
            token_counter[token] = 1
        else:
            token_counter[token] += 1
            
for sample in test_left_tokenized:
    for token in sample:
        if token not in token_counter:
            token_counter[token] = 1
        else:
            token_counter[token] += 1

for sample in test_right_tokenized:
    for token in sample:
        if token not in token_counter:
            token_counter[token] = 1
        else:
            token_counter[token] += 1
            
print("unique token count :", len(token_counter))

unique token count : 5824


In [9]:
# 根据词典，将数据转换成特征向量。
def encode_samples(tokenized_samples, vocab):
    features = []
    for sample in tokenized_samples:
        feature = []
        for token in sample:
            if token in vocab.token_to_idx:
                feature.append(vocab.token_to_idx[token])
            else:
                feature.append(0)
        features.append(feature)         
    return features

def pad_samples(features, maxlen=500, padding=0):
    padded_features = []
    for feature in features:
        if len(feature) > maxlen:
            padded_feature = feature[:maxlen]
        else:
            padded_feature = feature
            # 添加 PAD 符号使每个序列等长（长度为 maxlen ）。
            while len(padded_feature) < maxlen:
                padded_feature.append(padding)
        padded_features.append(padded_feature)
    return padded_features

In [10]:
vocab = text.vocab.Vocabulary(token_counter, unknown_token='<unk>', reserved_tokens=None)

left_texts_features = encode_samples(left_tokenized, vocab)
right_texts_featrues = encode_samples(right_tokenized, vocab)

left_padded_features = pad_samples(left_texts_features, 40, 0)
right_texts_featrues = pad_samples(right_texts_featrues, 40, 0)

test_left_features = encode_samples(test_left_tokenized, vocab)
test_right_featrues = encode_samples(test_right_tokenized, vocab)

test_left_padded_features = pad_samples(test_left_features, 40, 0)
test_right_texts_featrues = pad_samples(test_right_featrues, 40, 0)

In [11]:
num_validation_samples = int(0.2 * len(left_padded_features))

left_x_train = left_padded_features[:-num_validation_samples]
left_x_val = left_padded_features[-num_validation_samples:]

right_x_train = right_texts_featrues[:-num_validation_samples]
right_x_val = right_texts_featrues[-num_validation_samples:]

y_train = y[:-num_validation_samples]
y_val = y[-num_validation_samples:]

In [12]:
train_inputs = []
for i in range(len(left_x_train)):
    train_inputs.append([left_x_train[i], right_x_train[i]])
    
val_inputs = []
for i in range(len(left_x_val)):
    val_inputs.append([left_x_val[i], right_x_val[i]])

test_inputs = []
for i in range(len(test_left_padded_features)):
    test_inputs.append([test_left_padded_features[i], test_right_texts_featrues[i]])

In [13]:
ctx = mx.gpu()

train_features = nd.array(train_inputs, ctx=ctx)
train_label = nd.array(y_train, ctx)
val_features = nd.array(val_inputs, ctx=ctx)
val_label = nd.array(y_val, ctx)
test_featrues = nd.array(test_inputs, ctx=ctx)
# test_label = nd.array(test_label, ctx=ctx)

In [14]:
glove_embedding = text.embedding.CustomEmbedding(pretrained_file_path='data/wiki.es.vec', vocabulary=vocab)

D:\Anaconda3\envs\gluon\lib\site-packages\mxnet\contrib\text\embedding.py:278: UserWarning: At line 1 of the pre-trained text embedding file: token 985667 with 1-dimensional vector [300.0] is likely a header and is skipped.
  'skipped.' % (line_num, token, elems))


In [15]:
class Residual(nn.Block):
    def __init__(self, num_channels, use_1x1conv=False, strides=1, **kwargs):
        super(Residual, self).__init__(**kwargs)
        self.conv1 = nn.Conv1D(num_channels, kernel_size=3, padding=1, strides=strides)
        self.conv2 = nn.Conv1D(num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv1D(num_channels, kernel_size=1, strides=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm()
        self.bn2 = nn.BatchNorm()
        
        
    def forward(self, X):
        Y = nd.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        return nd.relu(Y + X)

In [16]:
num_outputs = 1
lr = 0.01
num_epochs = 100
batch_size = 100
embed_size = 300

(4, 3, 10)

In [ ]:
net = SentimentNet(vocab, embed_size, num_hiddens, num_layers, bidirectional)
net.initialize(init.Xavier(), ctx=ctx)
# 设置 embedding 层的 weight 为预训练的词向量。
net.embedding.weight.set_data(glove_embedding.idx_to_vec.as_in_context(ctx))
# 训练中不更新词向量（net.embedding中的模型参数）。
net.embedding.collect_params().setattr('grad_req', 'null')
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr})
loss = gloss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=True)